In [1]:
!pip install flask-ngrok
!pip install flask==0.12.2  
!pip install flask-cors
!pip install xgboost==1.4.2

     |████████████████████████████████| 83 kB 1.0 MB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4
     |████████████████████████████████| 166.7 MB 15 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [2]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok   

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,622 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://ngrok-agent.s3.amazonaws.com buster InRelease [7,470 B]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,490 kB]
Hit:12 http://ppa.

In [3]:
!ngrok authtoken 

NAME:
  authtoken - 

USAGE:
  ngrok authtoken TOKEN [flags]

DESCRIPTION: 

OPTIONS:
      --config string       save in this config file (default /root/.config/ngrok/ngrok.yml)
  -h, --help                help for authtoken
      --log string          path to log file, 'stdout', 'stderr' or 'false' (default "false")
      --log-format string   log record format: 'term', 'logfmt', 'json' (default "term")
      --log-level string    logging level: 'debug', 'info', 'warn', 'error', 'crit' (default "info")

ERROR:  accepts 1 arg(s), received 0


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request, redirect, url_for, flash, jsonify
from flask_cors import CORS, cross_origin
import pandas as pd
import xgboost as xgb
import pickle
import numpy as np

# __________________________________________________________________________
class EncoderExt(object):
    def __init__(self):
        self.label_encoder = LabelEncoder()
     
    def fit(self, data_list):
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_
        return self

    def transform(self, data_list):
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x == unique_item else x for x in new_data_list]
        return self.label_encoder.transform(new_data_list)
# __________________________________________________________________________



pickle_files = pickle.load(open("pickle_save_files.pkl", "rb"))
name_encoder, business_ecoder, cust_payment_terms_encoder, model = pickle_files

cols = ['cust_number', 'buisness_year', 'doc_id', 'converted_usd','business_code_enc', 'name_customer_enc', 
        'cust_payment_terms_enc','day_of_postingdate', 'month_of_postingdate', 'year_of_postingdate',
        'day_of_createdate', 'month_of_createdate', 'year_of_createdate', 'day_of_due', 'month_of_due', 'year_of_due']



def add_missing(row):
    temp = row
    temp["buisness_year"] = row["business_year"]
    if temp["invoice_currency"] == "CAD":
        temp["converted_usd"] = row["total_open_amount"]*0.7
    else:
         temp["converted_usd"] = row["total_open_amount"] 
    temp["day_of_postingdate"] = row["posting_date"]["day"]
    temp["month_of_postingdate"] = row["posting_date"]["month"]
    temp["year_of_postingdate"] = row["posting_date"]["year"]
    
    temp["day_of_createdate"] = row["document_create_date"]["day"]
    temp["month_of_createdate"] = row["document_create_date"]["month"]
    temp["year_of_createdate"] = row["document_create_date"]["year"]
    
    temp["day_of_due"] = row["due_in_date"]["day"]
    temp["month_of_due"] = row["due_in_date"]["month"]
    temp["year_of_due"] = row["due_in_date"]["year"]
   
    return temp


def return_df(rows):
    for row in rows:
        row = add_missing(row)
    df = pd.DataFrame(rows)
    df['cust_payment_terms_enc'] = cust_payment_terms_encoder.transform(df['cust_payment_terms'])
    df['business_code_enc'] = business_ecoder.transform(df['business_code'])
    df['name_customer_enc'] = name_encoder.transform(df['name_customer'])
    df = df[cols]
    return df

def return_predicted_date(due_in_date, delay_seconds):
    due_in_date = "{}-{}-{}".format(due_in_date["year"], due_in_date["month"], due_in_date["day"])
    due_in_date = pd.to_datetime(due_in_date)
    predicted_date = due_in_date + pd.Timedelta(seconds=delay_seconds)
    return predicted_date.strftime("%Y-%m-%d")

def funct(data):
    data_list = []
    json_df = return_df(data)
    result =  model.predict(json_df)
    for i in range(len(data)):
        temp = {}
        temp["sl_no"] = data[i]["id"]
        temp["predicted_payment_date"] = return_predicted_date(data[i]["due_in_date"], result[i])
        data_list.append(temp)
    return data_list
    


app = Flask(__name__)
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False
run_with_ngrok(app) 
CORS(app, support_credentials=True)

@app.route('/predict', methods=['POST','GET'])
def predict():
    data = funct(request.json)
    print(data)
    
    return jsonify(data)


@app.route('/')
def home():
    return render_template('index.html')


if __name__ == '__main__':
    app.run()



/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://313a-35-229-106-226.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


[2022-04-12 19:47:53,983] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1598, in dispatch_request
    r